In [ ]:
from google.colab import drive
import pandas as pd 
import gensim
from gensim.models import Word2Vec
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install openai

In [ ]:
import pandas as pd 
df=pd.read_csv('gdrive/My Drive/NIER-ICSE/data/test/dataWithAssociationValidation.csv')
df
df.dropna(inplace=True)
df

In [ ]:
dataframe = pd.DataFrame([], columns = ['sequence', 'package'])
package=df.iloc[0]['packageName']
seq=[]
for i, row in df.iterrows():
   
    if row['packageName']==package:
      seq.append('('+ row['classSource'] +','+ row['assocs'] +','+ row['classTarget'] +')')
      
    else: 
      dataframe=dataframe.append({'sequence': seq, 'package': package}, ignore_index=True)
      package=df.iloc[i]['packageName']
      seq=[]
      seq.append('('+ row['classSource'] +','+ row['assocs'] +','+ row['classTarget'] +')')

dataframe

In [ ]:
import random
dataframe = pd.DataFrame([], columns = ['sequence', 'package'])
package=df.iloc[0]['packageName']
seq=[]
for i, row in df.iterrows():
   
    if row['packageName']==package:
      seq.append('['+ row['classSource'] +','+ row['classTarget'] +'] => '+ row['assocs'] )
      
    else: 
      if len(seq)>3: 
        seq2= random.sample(seq, 3)
      else : 
        seq2=seq
      dataframe=dataframe.append({'sequence': seq2, 'package': package}, ignore_index=True)
      package=df.iloc[i]['packageName']
      seq=[]
      seq.append('['+ row['classSource'] +','+ row['classTarget'] +'] => '+ row['assocs'] )

dataframe

In [ ]:
d=dataframe.sample(7)
d

In [ ]:
data=""
for i, row in d.iterrows():
  data=data+ "\n" +str(row['sequence'])
  
data

###Prepare missing data

In [ ]:
import random
dataframe = pd.DataFrame([], columns = ['sequence', 'package','test'])
package=df.iloc[0]['packageName']
seq=[]
for i, row in df.iterrows():
   
    if row['packageName']==package:
      seq.append('['+ row['classSource'] +','+ row['classTarget'] +'] => '+ row['assocs'] )
      
    else: 
      test=seq[len(seq)-1]
      if len(seq)>3: 
        seq2= random.sample(seq[:len(seq)-1], 3)
      else : 
        seq2=seq
      dataframe=dataframe.append({'sequence': seq2, 'package': package, 'test' : test}, ignore_index=True)
      package=df.iloc[i]['packageName']
      seq=[]
      seq.append('['+ row['classSource'] +','+ row['classTarget'] +'] => '+ row['assocs'] )

dataframe

In [ ]:
#dataframe.to_csv("gdrive/My Drive/NIER-ICSE/data/test/AssociationTestData.csv")

In [ ]:
dataframe

In [ ]:
def predictRelation(seq, couple,temp=0.7):
    prompt ="predict association name: \n'[System,Author] => authors', '[Book,Chapter] => chaps', '[Chapter,Book] => itsbook'\n'[RoomStructure,RoomTypeHandling] => inherits from', '[Staff,SupportTicketReader] => inherits from', '[Staff,SupportTicketReader] => supportTicketReader'\n'[TA,Organisation] => organisation', '[Lecturer,Person] => inherits from', '[Organisation,CourseCoordinator] => courseCoordinator'\n'[And,Equatable] => right', '[Forall,Statement] => inherits from', '[Equality,Comparable] => right'\n'[Book,Person] => author', '[UoD,Person] => allPersons', '[Person,Loan] => loans'\n'[Country,Company] => companies', '[Neighborhood,NamedElement] => inherits from', '[Neighborhood,School] => schools'\n'[Chapter,Section] => sections', '[Selection,BodyElementContainer] => container', '[Schemadoc,Category] => category'\n'[employee , company] => worksIn, [manager, employee ] =>  supervises'\n'[person, Home] =>  owns','[car ,driver] => drives '\n'[personalCustomer, customer] => is', '[order,  customer] =>   places', '[employee, department] =>  worksIn'\n'[lion, meat] => eats', '[cat, milk] => drinks'\n"  +str(seq) +", " + str(couple)+ " =>" ,

    response = openai.Completion.create(
        model="text-davinci-002",
        prompt=  prompt,
        temperature=temp,
        max_tokens=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    res = response.choices[0].text
    #print( "[" +className +"],[ " + relatedClass+ "] =>" + res)

    return res
def predictTypeRelation(couples,temp=0.7):

    response = openai.Completion.create(
        model="text-davinci-002",
        prompt="What type of associations these concepts have: \n\nstudent , person => inheritance\ncomputer, cpu => composition\nplane , passenger => no \nperson , address => association \naccount , saving => inheritance\nBank, Atm => composition\nnurse , bank => no \nprofessor, student => association \nprofessor , person => inheritance\ncontact , address => composition\ncircle , bank => no \naccount , checking => inheritance \ncustomer , account => association\nbook , paper => composition\nbook , doctor => no \nlion , meat => association \nlion , salad => no \nmammal , dog => inheritance\ncustomer, order => association\naddressBook, contact   => composition\nBank, Atm => composition\nmanager, employee =>  association\nplane, manager =>    no\n" + str(couples) + '=> ',
        temperature=temp,
        max_tokens=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    res = response.choices[0].text
    return res

    

In [ ]:
def extractCouple(couples): 

  elements=couples[couples.find("[")+1:couples.find("]")]
  return elements.split(':')

In [ ]:
dataframe['couples']=dataframe['test'].apply(lambda df: extractCouple(df))

In [ ]:
predictRelation("['[Waggon,Waggon] => WaggonOrder_Waggon_role_successor', '[Station,Reservation] => Rto_Reservation_role_resTo', '[Reservation,Station] => RFrom_Station_role_fromStation']","['Reservation,Waggon']")

' RW'

In [ ]:
predictRelation(dataframe['sequence'][1],dataframe['couples'][1])

['ObjectFlow,ActivityEdge']


' source'

In [ ]:
predictTypeRelation(dataframe['couples'][1])


['ObjectFlow,ActivityEdge']


' no'

In [ ]:
dataframe['RelationName']=dataframe.apply(lambda df: predictRelation(df['sequence'],df['couples']),axis=1)

In [ ]:
dataframe['RelationName']

In [ ]:
dataframe['RElationType']=dataframe.apply(lambda df: predictTypeRelation(df['couples']),axis=1)

In [ ]:
dataframe

In [ ]:
dataframe.to_csv("gdrive/My Drive/NIER-ICSE/data/test/AssociationResults.csv")

##Ranking 

In [ ]:
dataframe=pd.read_csv("gdrive/My Drive/NIER-ICSE/data/test/AssociationResults.csv")

In [ ]:
from numpy import NaN
def predictRelation_(seq, couple):

    print(couple)
    results=[]
    prompt ="predict association name: \n'[System,Author] => authors', '[Book,Chapter] => chaps', '[Chapter,Book] => itsbook'\n'[RoomStructure,RoomTypeHandling] => inherits from', '[Staff,SupportTicketReader] => inherits from', '[Staff,SupportTicketReader] => supportTicketReader'\n'[TA,Organisation] => organisation', '[Lecturer,Person] => inherits from', '[Organisation,CourseCoordinator] => courseCoordinator'\n'[And,Equatable] => right', '[Forall,Statement] => inherits from', '[Equality,Comparable] => right'\n'[Book,Person] => author', '[UoD,Person] => allPersons', '[Person,Loan] => loans'\n'[Country,Company] => companies', '[Neighborhood,NamedElement] => inherits from', '[Neighborhood,School] => schools'\n'[Chapter,Section] => sections', '[Selection,BodyElementContainer] => container', '[Schemadoc,Category] => category'\n'[employee , company] => worksIn, [manager, employee ] =>  supervises'\n'[person, Home] =>  owns','[car ,driver] => drives '\n'[personalCustomer, customer] => is', '[order,  customer] =>   places', '[employee, department] =>  worksIn'\n'[lion, meat] => eats', '[cat, milk] => drinks'\n"  +str(seq) +", " + str(couple)+ " =>" ,
    try:
      for temp in [0.7, 0.8,0.9]:
        response = openai.Completion.create(
          model="text-davinci-002",
          prompt=  prompt,
          temperature=temp,
          max_tokens=1,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        res = response.choices[0].text
        results.append(res)
    except:
      print("oups")
      return float('NaN')
    return results
    #print( "[" +className +"],[ " + relatedClass+ "] =>" + res)

    
def predictTypeRelation_(couples,temp=0.7):
    print(couples)

    results=[]
    try:
      for temp in [0.7, 0.8,0.9]:
        response = openai.Completion.create(
          model="text-davinci-002",
          prompt="What type of associations these concepts have: \n\nstudent , person => inheritance\ncomputer, cpu => composition\nplane , passenger => no \nperson , address => association \naccount , saving => inheritance\nBank, Atm => composition\nnurse , bank => no \nprofessor, student => association \nprofessor , person => inheritance\ncontact , address => composition\ncircle , bank => no \naccount , checking => inheritance \ncustomer , account => association\nbook , paper => composition\nbook , doctor => no \nlion , meat => association \nlion , salad => no \nmammal , dog => inheritance\ncustomer, order => association\naddressBook, contact   => composition\nBank, Atm => composition\nmanager, employee =>  association\nplane, manager =>    no\n" + str(couples) + '=> ',
          temperature=temp,
          max_tokens=1,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
      )
        res = response.choices[0].text
        results.append(res)
    except:
      print("oups")
     
      return float('NaN')
    return results

    

In [ ]:
#dataframe[35:]['RelationType_rank']=dataframe[35:].apply(lambda df: predictTypeRelation_(df['couples']),axis=1)
dataframe[32:]['Rank']=dataframe[32:].apply(lambda df: predictRelation_(df['sequence'],df['couples']),axis=1)

In [ ]:
del dataframe['RelationName_rank']

In [ ]:
dataframe.to_csv("gdrive/My Drive/NIER-ICSE/data/test/PartialResultsAssociations.csv")

#Metrics

In [ ]:
dataframe=pd.read_csv("gdrive/My Drive/NIER-ICSE/data/test/PartialResultsAssociations.csv")

In [ ]:
del dataframe["Unnamed: 0.1"]

In [ ]:
def most_frequent(List):
    if len(List)>0:
      counter = 0
      num = List[0]
      
      for i in List:
          curr_frequency = List.count(i)
          if(curr_frequency> counter):
              counter = curr_frequency
              num = i
 
      return num
    return float(NaN)

In [ ]:
dataframe.fillna("[]",inplace=True)

In [ ]:
dataframe['selectedType']=dataframe.apply(lambda df: most_frequent(eval(df['RelationType_rank'])),axis=1)

In [ ]:
dataframe['selectedName']=dataframe.apply(lambda df: most_frequent(eval(df['Rank'])),axis=1)

In [ ]:
def extractAssociationName(text):
  return text.split("=>")[1]

In [ ]:
dataframe['AssociationName_GroundTruth']=dataframe["test"].apply(lambda x : extractAssociationName(x))

In [ ]:
def changeType(text):
  if text== " inheritance":
    print("yes")
    return "inherits from"
  return text


In [ ]:
def changeType_(text):
  if text== " inher":
    print("yes")
    return "inherits from"
  return text


In [ ]:
dataframe['RElationType'][1]

In [ ]:
dataframe['selectedName']=dataframe['selectedName'].apply(lambda x : changeType_(x))

In [ ]:
dataframe['selectedType']=dataframe['selectedType'].apply(lambda x : changeType(x))

In [ ]:
def checkSimilarity(text,Gt): 
  if text.strip()==Gt.strip(): 
    return 1
  return 0

In [ ]:
dataframe['results']=dataframe.apply(lambda x : checkSimilarity(x["selectedName"], x["AssociationName_GroundTruth"]), axis=1)

In [ ]:
del dataframe['RElationType']
del dataframe['RelationName']

In [ ]:
del dataframe['Unnamed: 0']

In [ ]:
dataframe

,sequence,package,test,couples,RelationType_rank,Rank,selectedType,selectedName,AssociationName_GroundTruth
0,"['[ActivityNode,ActivityGroup] => inGroup', '[...",FundamentalActivities,"[ActivityGroup,NamedElement] => inherits from","['ActivityGroup,NamedElement']","[' Inher', ' association', ' association']","[' group', ' super', ' group']",association,group,inherits from
1,"['[Pin,ObjectNode] => inherits from', '[Pin,Ac...",BasicActivities,"[ObjectFlow,ActivityEdge] => inherits from","['ObjectFlow,ActivityEdge']","[' no', ' no', ' association']","[' incoming', ' is', ' is']",no,is,inherits from
2,"['[CentralBufferNode,ActivityNode] => redefine...",IntermediateActivities,"[InterruptibleActivityRegion,ActivityGroup] =>...","['InterruptibleActivityRegion,ActivityGroup']","[' no', ' composition', ' no']","[' interrupt', ' interrupt', ' interrupt']",no,interrupt,inherits from
3,"['[LoopNode,InputPin] => input', '[LoopNode,Ac...",StructuredActivities,"[SequenceNode,StructuredActivityNode] => inher...","['SequenceNode,StructuredActivityNode']","[' no', '\n', ' no']","[' edge', ' edge', ' nested']",no,edge,inherits from
4,"['[ExpansionRegion,ActivityEdge] => incoming',...",ExtraStructuredActivities,"[ExpansionNode,ObjectNode] => inherits from","['ExpansionNode,ObjectNode']","[' no', ' no', "" ['""]","[' in', ' ', ' in']",no,in,inherits from
5,"['[People,Contact] => contacts', '[Office,Cont...",addressbook,"[BookVersion,AddressBook] => book","['BookVersion,AddressBook']","[' composition', "" ['"", ' no']","[' uses', ' uses', ' address']",composition,uses,book
6,"['[Article,Category] => category', '[Section,B...",article,"[HtmlFormatter,Formatter] => inherits from","['HtmlFormatter,Formatter']","[' inheritance', ' composition', ' inheritance']","[' form', ' form', ' form']",inherits from,form,inherits from
7,"['[Banker,Party] => inherits from', '[Bank,Con...",bank,"[OnlineTransaction,Transaction] => inherits from","['OnlineTransaction,Transaction']","[' inheritance', ' inheritance', ' association']","[' inher', ' online', ' transactions']",inherits from,inherits from,inherits from
8,"['[Unpublished,Author] => authors', '[Proceedi...",BibTeX,"[MasterThesis,ThesisEntry] => inherits from","['MasterThesis,ThesisEntry']","[' association', ' association', ' association']","[' inher', ' inher', ' inher']",association,inherits from,inherits from
9,"['[Book,Author] => auths', '[Book,Chapter] => ...",Books,"[Author,Book] => mybs","['Author,Book']","[' association', ' composition', ' association']","[' auth', ' writes', "" ['""]",association,auth,mybs


In [ ]:
dataframe.to_csv("gdrive/My Drive/NIER-ICSE/data/test/newAssociationResults.csv")

In [ ]:
dataframe=pd.read_csv("gdrive/My Drive/NIER-ICSE/data/test/AssociationResults.csv")